<a href="https://colab.research.google.com/github/csdnak/McPython/blob/master/SHAP_E_Text_to_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 322 (delta 29), reused 13 (delta 13), pack-reused 278
Receiving objects: 100% (322/322), 11.71 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [9]:
%cd shap-e

/content/shap-e


In [10]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-8m2465ls/clip_9846f4f0d8da44cca8a1540556b11f10
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-8m2465ls/clip_9846f4f0d8da44cca8a1540556b11f10
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl si

In [11]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [17]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:08<00:00, 107MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [21]:
batch_size = 4
guidance_scale = 15.0
prompt = "A helicopter"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [22]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAzADMzAAAAMzMzM2ZmZmaZZpmZZmZmmZlmmWaZmZmZmc…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAzADMzAAAAMwAzMzMzMzNmZmZmZmaZZmZmmWaZmZmZmW…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMwAzMzMzM2ZmM2ZmZplmZmaZZpmZZm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAzADMzAAAAMzMAMwAzMzMzM2ZmM2YzZjNmZmZmZmaZZj…

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)